In [1]:
import pyglstudy as gl
import numpy as np

In [2]:
def generate_group_lasso_data(
    n,
    p,
    n_groups,
    rho=0.1,
):
    Sigma = np.full((p, p), rho)
    np.fill_diagonal(Sigma, 1)
    X = np.random.normal(size=(n, p)) @ Sigma
    beta = np.random.normal(size=(p,))
    y = X @ beta + np.random.normal(size=(n,))

    A = (X.T @ X) / n
    r = (X.T @ y) / n

    order = np.arange(1, p) 
    groups = np.sort(np.random.choice(
        order, (n_groups-1,), replace=False,
    ))
    groups = np.concatenate([[0], groups, [p]], dtype=np.int32)
    group_sizes = groups[1:(n_groups+1)] - groups[:n_groups]
    groups = groups[:n_groups]
    
    return {
        "X": X,
        "beta": beta,
        "y": y,
        "A": A,
        "r": r,
        "groups": groups,
        "group_sizes": group_sizes,
    }

In [16]:
n = 100
p = 10
n_groups = 5

X, beta, y, A, r, groups, group_sizes = generate_group_lasso_data(n, p, n_groups).values()

A = np.asfortranarray(A)
alpha = 1.0
penalty = np.ones(p)
strong_set = np.arange(0, n_groups, dtype=np.int32)
strong_g1 = np.array([i for i in range(len(strong_set)) if group_sizes[strong_set[i]] == 1], dtype=np.int32)
strong_g2 = np.array([i for i in range(len(strong_set)) if group_sizes[strong_set[i]] > 1], dtype=np.int32)
assert((len(strong_g1) + len(strong_g2)) == n_groups)
strong_begins = np.array(np.cumsum(np.concatenate(
    [[0], np.array([group_sizes[i] for i in range(len(strong_set))], dtype=np.int32)],
), dtype=np.int32)[:-1])
strong_A_diag = np.array(np.concatenate(
    [
        np.diag(A)[groups[i] : (groups[i] + group_sizes[i])]
        for i in strong_set
    ]
))
lmdas = np.array([1, 0.5, 0.1, 0.01, 0.001])
max_cds = int(1e5)
thr = 1e-7
newton_tol = 1e-8
newton_max_iters = int(100)
rsq = 0.0
strong_beta = np.zeros((p,))
strong_grad = np.copy(r)
active_set = np.zeros((1,), dtype=np.int32)[:]
active_g1 = np.zeros((1,), dtype=np.int32)
active_g2 = np.zeros((1,), dtype=np.int32)
active_begins = np.zeros((1,), dtype=np.int32)
active_order = np.zeros((1,), dtype=np.int32)
is_active = np.zeros((p,), dtype=bool)

"""
arg0: numpy.ndarray[numpy.float64[m, n], flags.writeable, flags.f_contiguous]
arg1: numpy.ndarray[numpy.int32[m, 1], flags.writeable]
arg2: numpy.ndarray[numpy.int32[m, 1], flags.writeable]
arg3: float
arg4: numpy.ndarray[numpy.float64[m, 1], flags.writeable]
arg5: numpy.ndarray[numpy.int32[m, 1], flags.writeable]
arg6: numpy.ndarray[numpy.int32[m, 1], flags.writeable]
arg7: numpy.ndarray[numpy.int32[m, 1], flags.writeable]
arg8: numpy.ndarray[numpy.int32[m, 1], flags.writeable]
arg9: numpy.ndarray[numpy.float64[m, 1], flags.writeable]
arg10: numpy.ndarray[numpy.float64[m, 1], flags.writeable]
arg11: int
arg12: float
arg13: float
arg14: int
arg15: float
arg16: numpy.ndarray[numpy.float64[m, 1], flags.writeable]
arg17: numpy.ndarray[numpy.float64[m, 1], flags.writeable]
arg18: numpy.ndarray[numpy.int32[m, 1], flags.writeable]
arg19: numpy.ndarray[numpy.int32[m, 1], flags.writeable]
arg20: numpy.ndarray[numpy.int32[m, 1], flags.writeable]
arg21: numpy.ndarray[numpy.int32[m, 1], flags.writeable]
arg22: numpy.ndarray[numpy.int32[m, 1], flags.writeable]
arg23: numpy.ndarray[bool[m, 1], flags.writeable])
"""

gl.group_lasso(
    #A, 
    #groups,
    #group_sizes,
    #alpha,
    #penalty,
    #strong_set,
    #strong_g1,
    #strong_g2,
    #strong_begins,
    #strong_A_diag,
    #lmdas,
    #max_cds,
    #thr,
    #newton_tol,
    #newton_max_iters,
    #rsq,
    #strong_beta,
    #strong_grad,
    active_set,
    active_g1,
    active_g2,
    active_begins,
    active_order,
    is_active,
)

{}